In [1]:
# MLP for Pima Indians Dataset Serialize to JSON and HDF5

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import model_from_json
import numpy
import os
# fix random seed for reproducibility
numpy.random.seed(7)

# load pima indians dataset

import matplotlib.pyplot as plt
import pandas as pd
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
df = pd.read_csv('pima-indians-diabetes.data.csv', names=col_names)
# # load pima indians dataset
# dataset = numpy.loadtxt("pima-indians-diabetes.data.csv", delimiter=",")

# selection of relevant features
label = df['label']
df.drop('label', axis=1, inplace=True)

X, Y = df, label
df.describe()

# normalized data
df_norm = (df - df.mean()) / (df.max() - df.min())
X.describe()

/home/yash/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000


In [11]:
from keras import regularizers
from keras import optimizers
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

# create model
model = Sequential()

model.add(Dense(16, input_dim=8, kernel_initializer='glorot_uniform', activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.4))
model.add(Dense(8, kernel_initializer='glorot_uniform', activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(4, kernel_initializer='glorot_uniform', activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid'))

# Compile model
# lr=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', cooldown=0, min_lr=0)

model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999), metrics=['accuracy'])


filepath="./Models/model-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',period=10)


In [14]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.25)
print x_train.shape
from keras.callbacks import History 
history = History()

callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.1, verbose=1),checkpoint]

# Fit the model
history=model.fit(x_train, y_train, epochs=1200, batch_size=128, callbacks = callbacks, verbose=0,validation_data=(x_val, y_val))
# model.save('./Models/zeo_model.h5')

(576, 8)


In [13]:
import matplotlib.pyplot as plt

%matplotlib inline
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

KeyError: 'acc'

In [5]:
# evaluate the model
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


acc: 65.10%


In [6]:

# from keras.models import load_model

# model.save('./Models/my_model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# # returns a compiled model
# # identical to the previous one
# model = load_model('./Models/my_model.h5')

In [7]:
from keras.models import load_model
model1 = load_model('./Models/my_model.h5')

In [8]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                144       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 36        
_________________________________________________________________
dropout_3 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total para

In [10]:
for layer in model.layers: print(layer.get_config(), layer.get_weights())

'''
first_layer_weights = model.layers[0].get_weights()[0]
first_layer_biases  = model.layers[0].get_weights()[1]
second_layer_weights = model.layers[1].get_weights()[0]
second_layer_biases  = model.layers[1].get_weights()[1]
'''


({'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'distribution': 'uniform', 'scale': 1.0, 'seed': None, 'mode': 'fan_avg'}}, 'name': 'dense_1', 'kernel_constraint': None, 'bias_regularizer': None, 'bias_constraint': None, 'dtype': 'float32', 'activation': 'relu', 'trainable': True, 'kernel_regularizer': {'class_name': 'L1L2', 'config': {'l2': 0.0010000000474974513, 'l1': 0.0}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'units': 16, 'batch_input_shape': (None, 8), 'use_bias': True, 'activity_regularizer': None}, [array([[ 1.94273070e-01,  4.32489999e-02, -9.48631763e-02,
        -7.86108896e-02,  1.54529661e-01, -2.64978680e-05,
        -5.70242107e-01, -9.66447517e-02, -4.25938398e-01,
        -3.29996765e-01,  9.44690630e-02, -2.76835859e-01,
         8.94062072e-02, -2.90184677e-01,  9.94177088e-02,
         3.51620257e-01],
       [ 1.80379987e-01, -2.19213039e-01, -1.75371170e-01,
        -1.63408220e-02,  1.39807209e-01, -4.33813920e-03,
      

'\nfirst_layer_weights = model.layers[0].get_weights()[0]\nfirst_layer_biases  = model.layers[0].get_weights()[1]\nsecond_layer_weights = model.layers[1].get_weights()[0]\nsecond_layer_biases  = model.layers[1].get_weights()[1]\n'